In [2]:
import json
import re
import sys
import os
import ast

from tqdm import tqdm
from collections import Counter
from pathlib import Path
#from pandas import json_normalize #Deprecated? Also not used

In [3]:
import sys
import re, numpy as np, pandas as pd
from pprint import pprint

from pandas.io.json import json_normalize

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
import nltk
from gensim.utils import simple_preprocess#, lemmatize deprecetated
from gensim.models import CoherenceModel

from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt

from gensim.models import KeyedVectors

from nltk.corpus import stopwords
import gensim.downloader as api
import string
from sklearn.semi_supervised import LabelSpreading, LabelPropagation

/home/g/g01115/.local/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# **`Clean and Process Data `**

In [41]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

from nltk.tokenize import TweetTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer as VS

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/g/g01115/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
df = pd.read_json("Feature_Reduction.json", orient='records',lines=True)

df

,eventID,eventType,postID,postCategories,postPriority,text,entities,extended_entities,favorite_count,hashtagEntities,...,lang,matching_rules,mediaEntities,metadata,possibly_sensitive,retweet_count,retweeted_status,source,truncated,user
0,fireColorado2012,wildfire,212311994286620672,[MultimediaShare],Unknown,RT @CBSDenver: The copter is on the way to the...,"{'symbols': [], 'urls': [], 'hashtags': [{'tex...",None,0.0,None,...,en,None,None,None,NaN,2.0,"{'contributors': None, 'text': 'The copter is ...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",0.0,"{'location': 'Boulder, Colorado', 'default_pro..."
1,fireColorado2012,wildfire,217746356842926080,[MultimediaShare],Medium,RT @ColoradoRapids: Photo of #FlagStaffFire in...,"{'symbols': [], 'urls': [], 'hashtags': [{'tex...",None,0.0,None,...,en,None,None,None,NaN,11.0,"{'retweeted': False, 'in_reply_to_screen_name'...","<a href=""http://twitter.com/download/iphone"" r...",0.0,"{'location': 'Denver', 'default_profile': Fals..."
2,fireColorado2012,wildfire,217732012314861568,[FirstPartyObservation],Medium,2 wildfires in Boulder County. We can see smok...,"{'symbols': [], 'urls': [], 'hashtags': [], 'u...",None,0.0,None,...,en,None,None,None,NaN,0.0,None,"<a href=""http://twitter.com/download/android"" ...",0.0,"{'location': 'Beautiful Colorado', 'default_pr..."
3,fireColorado2012,wildfire,216961334129078272,[Discussion],Low,RT @Jon_G3: Seeing 1/3 of Colorado on fire mak...,"{'symbols': [], 'urls': [], 'hashtags': [{'tex...",None,0.0,None,...,en,None,None,None,NaN,7.0,"{'contributors': None, 'text': 'Seeing 1/3 of ...","<a href=""http://www.tweetcaster.com"" rel=""nofo...",0.0,"{'location': 'Salem, OR', 'default_profile': F..."
4,fireColorado2012,wildfire,212552860590813184,[MultimediaShare],Medium,RT @dhorning11: RT @LarimerCounty: #HighParkFi...,"{'symbols': [], 'urls': [{'expanded_url': 'htt...",None,0.0,None,...,en,None,None,None,0.0,1.0,None,"<a href=""https://about.twitter.com/products/tw...",0.0,"{'location': 'San Francisco', 'default_profile..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56025,tennesseeTornadoOutbreak2020,tornado,1235236359310368768,"[ThirdPartyObservation, Weather, Location, Mul...",Low,Hottest Google Search in 31.2 hrs. Nashville t...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",None,0.0,None,...,en,"[{'tag': 'terms', 'id': 386668493812155987, 'i...",None,None,0.0,0.0,None,"<a href=""https://trendsmetadata.com"" rel=""nofo...",1.0,"{'id': 290889806, 'id_str': '290889806', 'name..."
56026,tennesseeTornadoOutbreak2020,tornado,1235337290144239616,"[ThirdPartyObservation, Location, MultimediaSh...",Low,A live report is next on the Nashville tornado...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",None,0.0,None,...,en,"[{'tag': 'terms', 'id': 386668493812155987, 'i...",None,None,NaN,0.0,None,"<a href=""http://twitter.com/download/iphone"" r...",0.0,"{'id': 579146884, 'id_str': '579146884', 'name..."
56027,tennesseeTornadoOutbreak2020,tornado,1235258820139638784,"[ThirdPartyObservation, Weather, Location, Mul...",Low,Officials are still cleaning up after tornadoe...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",None,0.0,None,...,en,"[{'tag': 'terms', 'id': 386668493812155987, 'i...",None,None,NaN,0.0,None,"<a href=""http://www.socastdigital.com"" rel=""no...",0.0,"{'id': 829719163, 'id_str': '829719163', 'name..."
56028,tennesseeTornadoOutbreak2020,tornado,1235253249957126144,"[ThirdPartyObservation, Weather, Location, Mul...",Low,Putnam County: Cookeville area tornado victims...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",None,0.0,None,...,en,"[{'tag': 'terms', 'id': 386668493812155987, 'i...",None,None,0.0,0.0,None,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",0.0,"{'id': 825386050243682311, 'id_str': '82538605..."


In [42]:
# But first, read in stopwrods
enStop = stopwords.words('english')

# Skip stop words, retweet signs, @ symbols, and URL headers
stop_words = enStop +\
    ["http", "https", "rt", "@", ":", "t.co", "co", "amp", "&amp;", "...", "\n", "\r"]
stop_words.extend(string.punctuation)

In [43]:
local_tokenizer = TweetTokenizer()
def tokenizer_wrapper(text):
    return local_tokenizer.tokenize(text)

In [48]:
def count_twitter_objs(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned.
    
    Returns counts of urls, mentions, and hashtags.
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    #return(parsed_text.count('URLHERE'),parsed_text.count('MENTIONHERE'),parsed_text.count('HASHTAGHERE'))
    return(parsed_text)

def sent_to_words(sentences):
    for sent in sentences:
        sent = str(sent)
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = count_twitter_objs(sent)
        yield(sent)

In [49]:
data = df.text.values.tolist()
print(len(data))

data_words = list(sent_to_words(data))
data_words


56030


['RT The copter is on the way to the HASHTAGHERE. Well be airing it live throughout our coverage. You can also follow ...',
 'RT Photo of HASHTAGHERE in Boulder as seen from at 4pm MT. So many wildfires throughout Colorado rt now, h ...',
 '2 wildfires in Boulder County. We can see smoke to the north.',
 'RT Seeing 1/3 of Colorado on fire makes me wonder how certain political figures can argue cutting fire fighter numbers HASHTAGHERE',
 'RT RT HASHTAGHERE burn area map as of Monday night 10 p.m. URLHERE',
 'RT Evacuation order issued in Colorados High Park Fire - URLHERE',
 'RT HASHTAGHERE update URLHERE',
 'RT praying for the people, animals and entire communities affected by this HASHTAGHERE. truly unreal.',
 'New Mexico, Colorado fires burn out of control URLHERE',
 'Colorado, New Mexico forest fire crews gain ground: (Reuters) - Fire crews battling flames roaring through natio... URLHERE',
 'FEMA has authorized the use of federal funds to help with firefighting costs for the HASHTA

In [50]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [51]:
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in tokenizer_wrapper(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner']) #Changed en to be full name en_core_web_sm #might change to trf
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]
    return texts_out

In [52]:
print(len(data_words))

56030


In [53]:
data_ready = process_words(data_words)

In [13]:
#df.to_json('data_ready.json', orient='records', lines=True)

In [54]:
print(len(df.eventID), len(df.eventType), len(data_ready))
#eventID = df.eventID.values.tolist()
print(data_ready[:10])
#print(data_ready[:5])
#print(len(data_ready), len())

#I think you have to turn all empty lists into null

count = 0

for i in data_ready:
    if i == []:
        count += 1
        
print(count)

df['processed_text']=data_ready
df.to_json('processed_fr.json', orient='records', lines=True)

56030 56030 56030
[['copter', 'way', 'air', 'live', 'coverage', 'also', 'follow'], ['see', 'pm', 'many', 'wildfire'], ['wildfire', 'see', 'smoke', 'north'], ['see', 'fire', 'make', 'wonder', 'certain', 'political', 'figure', 'argue', 'cut', 'fire', 'fighter', 'number'], ['burn', 'area', 'map', 'night', 'urlhere'], ['order', 'issue'], ['update'], ['pray', 'people', 'animal', 'entire', 'community', 'affect', 'truly', 'unreal'], ['fire', 'burn', 'control'], ['forest', 'fire', 'crew', 'gain', 'ground', 'crew', 'battle', 'flame', 'roar', 'natio']]
1391


In [55]:
data = pd.read_json("processed_fr.json", orient='records',lines=True)
data

,eventID,eventType,postID,postCategories,postPriority,text,entities,extended_entities,favorite_count,hashtagEntities,...,matching_rules,mediaEntities,metadata,possibly_sensitive,retweet_count,retweeted_status,source,truncated,user,processed_text
0,fireColorado2012,wildfire,212311994286620672,[MultimediaShare],Unknown,RT @CBSDenver: The copter is on the way to the...,"{'symbols': [], 'urls': [], 'hashtags': [{'tex...",None,0.0,None,...,None,None,None,NaN,2.0,"{'contributors': None, 'text': 'The copter is ...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",0.0,"{'location': 'Boulder, Colorado', 'default_pro...","[copter, way, air, live, coverage, also, follow]"
1,fireColorado2012,wildfire,217746356842926080,[MultimediaShare],Medium,RT @ColoradoRapids: Photo of #FlagStaffFire in...,"{'symbols': [], 'urls': [], 'hashtags': [{'tex...",None,0.0,None,...,None,None,None,NaN,11.0,"{'retweeted': False, 'in_reply_to_screen_name'...","<a href=""http://twitter.com/download/iphone"" r...",0.0,"{'location': 'Denver', 'default_profile': Fals...","[see, pm, many, wildfire]"
2,fireColorado2012,wildfire,217732012314861568,[FirstPartyObservation],Medium,2 wildfires in Boulder County. We can see smok...,"{'symbols': [], 'urls': [], 'hashtags': [], 'u...",None,0.0,None,...,None,None,None,NaN,0.0,None,"<a href=""http://twitter.com/download/android"" ...",0.0,"{'location': 'Beautiful Colorado', 'default_pr...","[wildfire, see, smoke, north]"
3,fireColorado2012,wildfire,216961334129078272,[Discussion],Low,RT @Jon_G3: Seeing 1/3 of Colorado on fire mak...,"{'symbols': [], 'urls': [], 'hashtags': [{'tex...",None,0.0,None,...,None,None,None,NaN,7.0,"{'contributors': None, 'text': 'Seeing 1/3 of ...","<a href=""http://www.tweetcaster.com"" rel=""nofo...",0.0,"{'location': 'Salem, OR', 'default_profile': F...","[see, fire, make, wonder, certain, political, ..."
4,fireColorado2012,wildfire,212552860590813184,[MultimediaShare],Medium,RT @dhorning11: RT @LarimerCounty: #HighParkFi...,"{'symbols': [], 'urls': [{'expanded_url': 'htt...",None,0.0,None,...,None,None,None,0.0,1.0,None,"<a href=""https://about.twitter.com/products/tw...",0.0,"{'location': 'San Francisco', 'default_profile...","[burn, area, map, night, urlhere]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56025,tennesseeTornadoOutbreak2020,tornado,1235236359310368768,"[ThirdPartyObservation, Weather, Location, Mul...",Low,Hottest Google Search in 31.2 hrs. Nashville t...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",None,0.0,None,...,"[{'tag': 'terms', 'id': 386668493812155987, 'i...",None,None,0.0,0.0,None,"<a href=""https://trendsmetadata.com"" rel=""nofo...",1.0,"{'id': 290889806, 'id_str': '290889806', 'name...","[hrs, tornado, tornado, tornado, tenness]"
56026,tennesseeTornadoOutbreak2020,tornado,1235337290144239616,"[ThirdPartyObservation, Location, MultimediaSh...",Low,A live report is next on the Nashville tornado...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",None,0.0,None,...,"[{'tag': 'terms', 'id': 386668493812155987, 'i...",None,None,NaN,0.0,None,"<a href=""http://twitter.com/download/iphone"" r...",0.0,"{'id': 579146884, 'id_str': '579146884', 'name...","[live, report, tornado]"
56027,tennesseeTornadoOutbreak2020,tornado,1235258820139638784,"[ThirdPartyObservation, Weather, Location, Mul...",Low,Officials are still cleaning up after tornadoe...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",None,0.0,None,...,"[{'tag': 'terms', 'id': 386668493812155987, 'i...",None,None,NaN,0.0,None,"<a href=""http://www.socastdigital.com"" rel=""no...",0.0,"{'id': 829719163, 'id_str': '829719163', 'name...","[official, still, clean, tornado, hit, part]"
56028,tennesseeTornadoOutbreak2020,tornado,1235253249957126144,"[ThirdPartyObservation, Weather, Location, Mul...",Low,Putnam County: Cookeville area tornado victims...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",None,0.0,None,...,"[{'tag': 'terms', 'id': 386668493812155987,

# Generating Similarity Scores and Matrix
### **Mean and Cosine Similarity(each event with all other event-types)**


In [56]:
import pickle
#import logging
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.pipeline import make_pipeline
from scipy import sparse
import numpy as np
import os.path
import re
from tqdm import tqdm
import warnings
import pandas as pd

In [57]:
data['l'] = data.apply(lambda row: len(row['processed_text']), axis=1)
data= data.query("l >1")
data.drop(columns=['l'], inplace=True)
data

,eventID,eventType,postID,postCategories,postPriority,text,entities,extended_entities,favorite_count,hashtagEntities,...,matching_rules,mediaEntities,metadata,possibly_sensitive,retweet_count,retweeted_status,source,truncated,user,processed_text
0,fireColorado2012,wildfire,212311994286620672,[MultimediaShare],Unknown,RT @CBSDenver: The copter is on the way to the...,"{'symbols': [], 'urls': [], 'hashtags': [{'tex...",None,0.0,None,...,None,None,None,NaN,2.0,"{'contributors': None, 'text': 'The copter is ...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",0.0,"{'location': 'Boulder, Colorado', 'default_pro...","[copter, way, air, live, coverage, also, follow]"
1,fireColorado2012,wildfire,217746356842926080,[MultimediaShare],Medium,RT @ColoradoRapids: Photo of #FlagStaffFire in...,"{'symbols': [], 'urls': [], 'hashtags': [{'tex...",None,0.0,None,...,None,None,None,NaN,11.0,"{'retweeted': False, 'in_reply_to_screen_name'...","<a href=""http://twitter.com/download/iphone"" r...",0.0,"{'location': 'Denver', 'default_profile': Fals...","[see, pm, many, wildfire]"
2,fireColorado2012,wildfire,217732012314861568,[FirstPartyObservation],Medium,2 wildfires in Boulder County. We can see smok...,"{'symbols': [], 'urls': [], 'hashtags': [], 'u...",None,0.0,None,...,None,None,None,NaN,0.0,None,"<a href=""http://twitter.com/download/android"" ...",0.0,"{'location': 'Beautiful Colorado', 'default_pr...","[wildfire, see, smoke, north]"
3,fireColorado2012,wildfire,216961334129078272,[Discussion],Low,RT @Jon_G3: Seeing 1/3 of Colorado on fire mak...,"{'symbols': [], 'urls': [], 'hashtags': [{'tex...",None,0.0,None,...,None,None,None,NaN,7.0,"{'contributors': None, 'text': 'Seeing 1/3 of ...","<a href=""http://www.tweetcaster.com"" rel=""nofo...",0.0,"{'location': 'Salem, OR', 'default_profile': F...","[see, fire, make, wonder, certain, political, ..."
4,fireColorado2012,wildfire,212552860590813184,[MultimediaShare],Medium,RT @dhorning11: RT @LarimerCounty: #HighParkFi...,"{'symbols': [], 'urls': [{'expanded_url': 'htt...",None,0.0,None,...,None,None,None,0.0,1.0,None,"<a href=""https://about.twitter.com/products/tw...",0.0,"{'location': 'San Francisco', 'default_profile...","[burn, area, map, night, urlhere]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56024,tennesseeTornadoOutbreak2020,tornado,1235274115340107776,"[ThirdPartyObservation, Weather, MultimediaSha...",Low,Discounts?\n\nLoretta Lynn offers support to N...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",None,0.0,None,...,"[{'tag': 'terms', 'id': 386668493812155987, 'i...",None,None,0.0,0.0,None,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",1.0,"{'id': 748477502, 'id_str': '748477502', 'name...","[discount, offer, support, tornado, victim, di..."
56025,tennesseeTornadoOutbreak2020,tornado,1235236359310368768,"[ThirdPartyObservation, Weather, Location, Mul...",Low,Hottest Google Search in 31.2 hrs. Nashville t...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",None,0.0,None,...,"[{'tag': 'terms', 'id': 386668493812155987, 'i...",None,None,0.0,0.0,None,"<a href=""https://trendsmetadata.com"" rel=""nofo...",1.0,"{'id': 290889806, 'id_str': '290889806', 'name...","[hrs, tornado, tornado, tornado, tenness]"
56026,tennesseeTornadoOutbreak2020,tornado,1235337290144239616,"[ThirdPartyObservation, Location, MultimediaSh...",Low,A live report is next on the Nashville tornado...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",None,0.0,None,...,"[{'tag': 'terms', 'id': 386668493812155987, 'i...",None,None,NaN,0.0,None,"<a href=""http://twitter.com/download/iphone"" r...",0.0,"{'id': 579146884, 'id_str': '579146884', 'name...","[live, report, tornado]"
56027,tennesseeTornadoOutbreak2020,tornado,1235258820139638784,"[ThirdPartyObservation, Weather, Location, Mul...",Low,Officials are still cleaning up after tornadoe...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",None,0.0,None,...,"[{'tag': 'terms', 'id': 3866684938121

In [ ]:
#This would run through the ssh oopsy, do not run
#pip install -U sentence-transformers

In [58]:
# generate sentnece embedding
class SBERT:

    def __init__(self, lang="en"):
        from sentence_transformers import SentenceTransformer
        self.name = "SBERT"
        if lang == "fr":
            self.model = SentenceTransformer(
                "/home/bmazoyer/Dev/pytorch_bert/output/sts_fr_long_multilingual_bert-2019-10-01_15-07-03")
        elif lang == "en":
            self.model = SentenceTransformer(
                # "bert-large-nli-stsb-mean-tokens"
                "roberta-large-nli-stsb-mean-tokens"
            )
# roberta-large-nli-stsb-mean-tokens
    def compute_vectors(self, data):
        data["text"] = data.text.str.slice(0, 500)
        vectors = np.array(self.model.encode(data.text.tolist()))
        return vectors

In [59]:
sbert=SBERT()

In [ ]:
v=sbert.compute_vectors(data)
data['sbert_emb']=[item for item in v]

<ipython-input-58-8d8bf9cebbb9>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["text"] = data.text.str.slice(0, 500)


In [ ]:
filename = 'fr_pickle'
outfile = open(filename,'wb')
pickle.dump(data,outfile)
outfile.close()